Pada tahapan kali ini saya ingin melakukan A/B testing untuk melihat adakan perbedaan signifikan antara 2 kelompok yaitu Kelompok: А (kontrol), B (corong pembayaran yang baru), dengan tujuan melihat perubahan terkait pengenalan sistem rekomendasi yang telah ditingkatkan, tujuannya melihat apakah ada perbedaan pengunjung yang signifikan sebelum nantinya di buat keputusan apakah memnggunakan corong pembayaran yang baru atau tidak. Ini suatu tahapan yang penting dilakukan karena akan menguji pelanggan yang ada di website kita apakah nyaman menggunakan sistem baru atau menggunakan sistem yang lama, ini dilihat dari jumlah total user_id(pengunjung) yang mengunjungi dan menggunakan sistem yang telah dibagi 2 kelompok, jika ada perbedaan yang signigikan antara 2 kelompok bisa dikatkan ada kelompok yang pengunjungnya nyaman menggunakan suatu sistem dan satunya tidak, jika perbedaan tidak signigikan artinya kebiasaan atau tingkat kenyamanan pengguna dalam 2 kelompok di suatu website dikatakan tidak berbeda jauh atau relatif sama.

Untuk core data penting yang saya pakai untuk analisis adalah new_user_event disini terlihat user baru yang melakukan aktivitas apa saja atau melakukan event apa saja, nantinya tidak semua data ini saya gunakan. Saya akan menggunakan data user yang mendaftar sebelum 22 desember 2020, karena website ini berhenti menerima pendaftar baru saat 21 desember 2020. Lalu untuk melakukan uji A/B test user_id baru ini dibagi menjadi 2 kelompok yang datanya sudah ada di test_participant karena tidak semua user_id di dataset new_user_event berpartisipasi dan untuk pembagian kelompok A atau kelompok B sudah tertera di dataset test_participant, dan saya nantinya akan menggabunggkan 2 data set ini (new_user_event dan test_participant) untuk melihat user_id mana yang akan ikut testing perubahan terkait pengenalan sistem rekomendasi yang telah ditingkatkan, langkah analisanya saya akan melihat tipe datanya apakah sudah sesuai atau belum untuk dianalisa,jika belum saya akan menggantinya setelah itu akan menghilangkan nilai NaN dan nilai duplikat. Setelah proses eskplorasi data selesai saya mulai masuk ke tujuan utama A/B testing yang pertama dilakukan adalah melihat corong total pengguna yang melakukan event dalam toko daring ini, setelah itu membagi berdasarkan event yang dilakukan oleh user_id tujuannya untuk melihat event yang dilakukan ada berapa user_id yang melakukannya. Setelah itu semua dilakukan saya menggunakan data user yang melakukan eksperiment "recommender_system_test" dan setelah itu melihat apakah ada user_id yang tergabung dalam kelompok uji lebih dari 1 di waktu yang sama (tergabung dalam grup A dan B sekaligus) jika ada maka tidak akan diikutkan test, setelah data siap mulai melakukan A/B testing, yang pertama saya lihat nanti adalah perbandingn rata-rata per-event yang dilakukan oleh user_id yang berada di kelompok A dan B untuk melihat dari sejumlah user_id yang berpartisipasi test yang telah dibagi ke kelompok A dan B untuk masing masing event yang ada di sini berapa persentasenya. Setelah itu saya melakukan z-test untuk melihat apakah ada perbedaan signigikan antara user yang melakukan event di kelompok A dan kelompok B untuk melihat perilaku mereka


## Eksplorasi data

### Konversi data

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
import math as mt

Memasukkan library yang diperlukan

In [2]:
calendar_events = pd.read_csv('/datasets/ab_project_marketing_events_us.csv')
user_sign_up = pd.read_csv('/datasets/final_ab_new_users_upd_us.csv')
new_user_event = pd.read_csv('/datasets/final_ab_events_upd_us.csv')
test_participant = pd.read_csv('/datasets/final_ab_participants_upd_us.csv')

Memuat Data Set yang diperlukan

In [3]:
calendar_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [4]:
calendar_events['start_dt'] = pd.to_datetime(calendar_events['start_dt'])
calendar_events['finish_dt'] = pd.to_datetime(calendar_events['finish_dt'])

melihat tipe data dan mengecek apakah ada nilai yang hilang pada dataset calendar_events, lalu disini saya mengubah tipe data pada columns start_dt dan finish_dt karena tipe datanya kurang tepat, saya mengubah tipe datanya menjadi datetime agar nantinya bisa dilakukan operasi penanggalan pada 2 column data tersebut

In [5]:
user_sign_up.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [6]:
user_sign_up['first_date'] = pd.to_datetime(user_sign_up['first_date'])

melihat tipe data dan mengecek apakah ada nilai null pada dataset user_sign_up dan saya mengubah tipe data pada data first date menjadi tipe date time

In [7]:
new_user_event.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [8]:
new_user_event['event_dt'] = pd.to_datetime(new_user_event['event_dt'] )

In [9]:
new_user_event['details']=new_user_event['details'].fillna(value='none')

In [10]:
new_user_event['date'] = new_user_event['event_dt'].dt.date 
new_user_event['date'] = pd.to_datetime(new_user_event['date'])
new_user_event

,user_id,event_dt,event_name,details,date
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99,2020-12-07
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99,2020-12-07
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99,2020-12-07
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99,2020-12-07
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99,2020-12-07
...,...,...,...,...,...
423756,245E85F65C358E08,2020-12-30 19:35:55,login,none,2020-12-30
423757,9385A108F5A0A7A7,2020-12-30 10:54:15,login,none,2020-12-30
423758,DB650B7559AC6EAC,2020-12-30 10:59:09,login,none,2020-12-30
423759,F80C9BDDEA02E53C,2020-12-30 09:53:39,login,none,2020-12-30


melihat tipe data dan mengecek apakah ada nilai yang hilang pada tipe data new_user_event, dataset ini berisi pengguna baru yang melakukan event dan waktu melakukan eventnya, untuk detail adalah detail event yang pelanggan itu lakukan, disini teradapat nilai yang hilang pada detail, detail ini menjelaskan data tambahan misalnya jika pelanggan itu melakukan purchase harga yang dikeluarkan berapa, karena selain peristiwa lain selain purchase dianggap tidak perlu detail tambahan maka nilainya tidak ada maka disini saya menggantinya dengan 'none', maksudnya karena tidak perlu informasi tambahan. Pada tipe data saya ganti yang menjelaskan tanggal pembelian menjadi datetime, lalu saya menambahkan kolom baru yaitu date, column ini berisi tanggal saja tanpa detail waktu event yang pelanggan itu lakukan

### Mengecek duplikasi data

In [11]:
new_user_event.duplicated().sum()

0

In [12]:
calendar_events.duplicated().sum()

0

In [13]:
user_sign_up.duplicated().sum()

0

In [14]:
test_participant.duplicated().sum()

0

mengecek apakah ada duplikasi data pada dataset yang dimuat dan hasilnya tidak ada data yang duplikat

## analisis data eksploratif

In [15]:
new_user_event['event_name'].unique()

array(['purchase', 'product_cart', 'product_page', 'login'], dtype=object)

melihat peristiwa apa saja yang ada pada data set

In [16]:
filtered_user = new_user_event[new_user_event['date']<= '2020-12-21']

disini saya melakukan pembatasan penggunaan data, saya menggunakan data pelanggan yang mendaftar sebelum 22 desember karena disini website berhenti menerima pengguna baru pada  21-12-2020, jadi setelah tanggal tersebut datanya tidak terpakai

### Konversi di berbagai tahapan corong

In [17]:
new_participant_data = filtered_user.merge(test_participant,on='user_id')
new_participant_data = new_participant_data[new_participant_data['ab_test']=='recommender_system_test'] 
new_participant_data

,user_id,event_dt,event_name,details,date,group,ab_test
9,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99,2020-12-07,A,recommender_system_test
10,831887FE7F2D6CBA,2020-12-09 02:19:17,purchase,99.99,2020-12-09,A,recommender_system_test
11,831887FE7F2D6CBA,2020-12-07 06:50:30,product_cart,none,2020-12-07,A,recommender_system_test
12,831887FE7F2D6CBA,2020-12-08 10:52:27,product_cart,none,2020-12-08,A,recommender_system_test
13,831887FE7F2D6CBA,2020-12-09 02:19:17,product_cart,none,2020-12-09,A,recommender_system_test
...,...,...,...,...,...,...,...
73655,928364C4C9F13FA8,2020-12-21 17:17:34,login,none,2020-12-21,B,recommender_system_test
73658,1484BBF124DB1B18,2020-12-21 17:51:44,login,none,2020-12-21,A,recommender_system_test
73660,BEF16764A13AEC34,2020-12-21 03:49:49,login,none,2020-12-21,B,recommender_system_test
73662,1E6B9298415AA97A,2020-12-21 18:20:12,login,none,2020-12-21,B,recommender_system_test


menggabungkan dataset new_user_event dan  test_participant menggunakan metode merge berdasarkan user_id, gunanya disini saya ingin mengetahui user_id baru yang di dataset new_user_event yang mengikuti tes yang letaknya di tes_participant, nantinya user_id ini diketahui dia berada di grup mana dan jenis ab_test yang dia lakukan itu apa dan dataset ini nantinya digunakan untuk melakukan A/B testing yang nantinya untuk melihat sample yang ada di grup A atau B ada perbedaan perilaku atau tidak.

In [18]:
user_per_event = new_participant_data.groupby('event_name').size().reset_index()
user_per_event.loc[[0,2,1,3],:]

,event_name,0
0,login,8001
2,product_page,4917
1,product_cart,2351
3,purchase,2352


### cek  pengguna yang berada di kedua sampel

In [19]:
check_user= new_participant_data.groupby('user_id')['group'].nunique().reset_index()
check_user[check_user['group']==2]

#a[a['ab_test']==2]

,user_id,group


pada tahap ini saya melihat apakah ada user_id yang masuk ke kelompok uji A dan B secara sekaligus, jika ternyata ada nantinya user_id ini tidak boleh ikut kelompok uji karena dia berada di 2 kelompok dalam 1 waktu, sehingga hasilnya akan mempengaruhi keakuratan di setiap kelompok uji

### pendistribusian jumlah peristiwa berdasarkan hari

In [20]:
new_participant_data.groupby('date')['event_name'].count().reset_index()

,date,event_name
0,2020-12-07,709
1,2020-12-08,593
2,2020-12-09,746
3,2020-12-10,613
4,2020-12-11,542
5,2020-12-12,558
6,2020-12-13,504
7,2020-12-14,1355
8,2020-12-15,1316
9,2020-12-16,1454


terlihat jumlah pendistribusi peristiwa berdasarkan hari jumlahnya berbeda, peak nya terjadi di tanggal

## Evaluasi hasil A/B testing

In [21]:
total_user_per_event_group = new_participant_data.pivot_table(index = 'event_name',columns='group',values='user_id',aggfunc='nunique')
#total_user_per_event_group = total_user_per_event_group.iloc[[0,2,1,3],:].reset_index()
total_user_per_event_group

group,A,B
event_name,,
login,2747,927
product_cart,824,255
product_page,1780,523
purchase,872,256


Pada tahap ini saya ingin melihat masing masing per-grup ada berapa user yang melakukan event pada toko daring dan melihat pebedaan konversinya pada masing masing grup a dan b, nantinya untuk melihat ada tidaknya pebedaan konversi dari satu event ke event lain pada masing masing grup menggunakan jumlah ini juga

In [22]:
total_user_group = new_participant_data.groupby('group')['user_id'].nunique()
total_user_group

group
A    2747
B     928
Name: user_id, dtype: int64

In [23]:
def tes(convert1,convert2,total1,total2,alpha=0.05):
    p1 = convert1/total1
    p2 = convert2/total2
    
    combine = (convert1+convert2)/(total1+total2)
    
    difference = p1-p2
    
    zvalues = difference/mt.sqrt(combine*(1-combine)*(1/total1 + 1/total2))
    
    distr = st.norm(0,1)
    
    pvalue = (1-distr.cdf(abs(zvalues)))*2
    
    #print('pvalue =',  pvalue)
    print("pvalue :{:.2f}".format(pvalue))
    
    if (pvalue < alpha) :
        print("menolak H0 : ada  perbedaan statistik di antara proporsinya, antara 2 kelompok")
    else:
        print("gagal menolak H0 : tidak ada  perbedaan statistik di antara proporsinya, antara 2 kelompok")


In [24]:
tes(total_user_per_event_group.loc['login','A'],
    total_user_per_event_group.loc['login','B'],
    total_user_group.loc['A'],
    total_user_group.loc['B'])

pvalue :0.09
gagal menolak H0 : tidak ada  perbedaan statistik di antara proporsinya, antara 2 kelompok


fungsi ini saya gunakan untuk mencari adakah perbedaan signifikan antara pengguna yang ada di kelompok A(kontorl) dan kelompok B(corong pembayaran baru) menggunakan metode z_test caranya langkahnya :

1. saya mencari sukses rate pada masing masing kelompok uji antar A dan B 

2. setelah itu mencari nilai succes rate gabungan kelompou uji A dan B

3. selanjutnya saya mencari perbedaan succes rate koversi untuk setiap kelompok uji, dengan melakukan tahap ini kita ingin mengetahui apakah ada perbedaan signifikan antara success rate konversion antara kelompok uji 246 dan 247

4. pada variable zvalues saya ingin mengetahui berapa besar perbedaan success rate menggunakan metode standart deviasi dari distribusi normal standart dengan menggunakan zvalues saya ingin mengetahui jumlah perbedaan success rate 2 kelompok dalam satuan standart deviasi

5. pada bagian distr saya menggunakan untuk menentukan nilai p_value yang digunakan untuk menentukan hasil hipotesis, nilai dari distr(distribusi normal standard) digunakan untuk mencari pvalue dengan menggunakan fungsi cdf(fungsi disribusi cumulative)

6. setelah dapat nilai p_value saya membandingkan nya apakah nilai p_value yang dihasilkan kurang atau lebih dari nilai alpha yang sudah saya set sebelumnya diatas, jika nilainya kurang dari alpha maka kita bisa menolak H0 dan jika nilai lebih dari alpha kita bisa menolak H0

In [25]:
def testing_event(total_user_per_event_group,total_user_grup,event_name,group1,group2):
    groupA = total_user_per_event_group.loc[event_name,group1]/total_user_group.loc[group1],
    groupB = total_user_per_event_group.loc[event_name,group2]/total_user_group.loc[group2],
    print(f'{groupA} with {event_name} in group {group1}'),
    print(f'{groupB} with {event_name} in group {group2}'),
    tes(total_user_per_event_group.loc[event_name,group1],
        total_user_per_event_group.loc[event_name,group2],
        total_user_group.loc[group1],
        total_user_group.loc[group2])

In [ ]:
total_user_per_event_group.loc['login']

pada fungsi ini saya ingin mengetahui jumlah rata rata perbandingan antara grup A dan grup B untuk setiap event yang terjadi digrup A dan grup B tujuannya adalah untuk melihat adakan perbedaan signifikand dari setiap peristiwa/event yang dilakukan user di masing masing kelompok A dan B 

hal pertama yang dilakukan adalah mencari persentase dimasing masing kelompok A dan B dengan membagi jumlah total user yang melakukan event yang sedang terjadi dengan total user yang ada di masing masing kelompok

In [26]:
for event_name in total_user_per_event_group.index:
    testing_event(total_user_per_event_group,total_user_group,event_name,'A','B')
    
    print()
   

(1.0,) with login in group A
(0.9989224137931034,) with login in group B
pvalue :0.09
gagal menolak H0 : tidak ada  perbedaan statistik di antara proporsinya, antara 2 kelompok

(0.2999635966508919,) with product_cart in group A
(0.27478448275862066,) with product_cart in group B
pvalue :0.15
gagal menolak H0 : tidak ada  perbedaan statistik di antara proporsinya, antara 2 kelompok

(0.6479796141244994,) with product_page in group A
(0.5635775862068966,) with product_page in group B
pvalue :0.00
menolak H0 : ada  perbedaan statistik di antara proporsinya, antara 2 kelompok

(0.3174372042227885,) with purchase in group A
(0.27586206896551724,) with purchase in group B
pvalue :0.02
menolak H0 : ada  perbedaan statistik di antara proporsinya, antara 2 kelompok



pada tahapan diatas saya melakukan pengujian untuk setiap peristiwa yang terjadi di kedua kelompok A dan B, untuk peristiwa login todal ada berbedaan signifikan antara kedua kelompoknya, untuk peristiwa product_cart juga tidak ada perbedaan signifikan, sedangkan untuk peristiwa product_page dan purchase ada perbedaan antara user di kelompok A dan kelompok B, mungkin perbedaannya ada di jumlah pembelian yang dihabiskan masing-masing user di kelompok uji, bisa jadi user kelompok A lebih banyak menghabiskan uang dibanding user kelompok B pada event purchase

## Kesimpulan

dengan serangkaian test A/B testing yang telah diuji coba hasilnya untuk kelompok A dan kelompok B tidak ada perbedaan statistik diantara proporsinya, kemungkinan untuk setiap corong peristiwa masing masing kelompok konversi dari satu peristiwa ke peristiwa nilainya hampir sama, jadi setiap peristiwa seteleah login , product_page → product_cart → purchase konvenrsi untuk dari kelompok A atau kelompok B sama dari satu peristiwa jumlah user_id yang melakukan event semakin mengecil sama seprti corong